In [34]:
import numpy as np
import pandas as pd

In [35]:
def ngrams(input_list, n):
  return zip(*[input_list[i:] for i in range(n)])

def jCDistance(A,B):
	#A intersect B / (A union B) 
	#A: list
	#B: list

	deno = set(A) & set(B)
	nom = set(A) | set(B)

	jC = len(deno) / float(len(nom))
	return jC

def featureGenerate(title1, title2):
    t11 = ngrams(title1,1)
    t21 = ngrams(title2,1)

    jc1 = jCDistance(t11,t21)

    result = jc1
    return result

## Train Feature

In [36]:
train_attr = pd.read_csv('train_attr.csv')
test_attr = pd.read_csv('test_attr.csv')

In [37]:
train_attr.head(3)

,id,product_uid,product_title,search_term,product_description,relevance,productList,searchList,desList,brand,color,material
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,"Not only do angles make joints stronger, they ...",3.0,simpson strong-ti 12-gaug angl,angl bracket,angl brand joint strong besid suppli consist h...,simpson strong-ti,NaN,startl steel
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,"Not only do angles make joints stronger, they ...",2.5,simpson strong-ti 12-gaug angl,liter bracket,angl brand joint strong besid suppli consist h...,simpson strong-ti,NaN,startl steel
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,BEHR Premium Textured DECKOVER is an innovativ...,3.0,behr premium textur deck 1-gal sc-141 tugboat ...,deck,behr premium textur deckov advanc solid color ...,behr premium textur deck,tugboat,NaN


In [38]:
brandDis = []
colorDis = []
materialDis = []
for i in range(len(train_attr['searchList'])):
    if type(train_attr['searchList'][i]) == str:
        if type(train_attr['brand'][i]) == str:
            brandDis.append(featureGenerate(train_attr['searchList'][i].split(),train_attr['brand'][i].split()))
        else:
            brandDis.append(0)
        if type(train_attr['color'][i]) == str:
            colorDis.append(featureGenerate(train_attr['searchList'][i].split(),train_attr['color'][i].split()))
        else:
            colorDis.append(0)
        if type(train_attr['material'][i]) == str:
            materialDis.append(featureGenerate(train_attr['searchList'][i].split(),train_attr['material'][i].split()))
        else:
            materialDis.append(0)
    else:
        brandDis.append(0)
        colorDis.append(0)
        materialDis.append(0)

In [39]:
train_attr['brandDis'] = pd.Series(brandDis)
train_attr['colorDis'] = pd.Series(colorDis)
train_attr['materialDis'] = pd.Series(materialDis)

In [40]:
train_attr1 = train_attr.iloc[:,[0,12,13,14]]

In [44]:
train_attr1.head(2)

,id,brandDis,colorDis,materialDis
0,2,0,0,0
1,3,0,0,0


In [41]:
train_feature = pd.read_csv('features_tfidf_d_train.csv')

In [45]:
train_feature.head(2)

,id,relevance,ts1,ts2,ts3,td1,td2,sd1,sd2,sd3,...,td1_tfidf,td2_tfidf,sd1_tfidf,sd2_tfidf,sd3_tfidf,sd4_tfidf,sd5_tfidf,sd6_tfidf,sd7_tfidf,sd8_tfidf
0,2,3.0,0.2,0,0,0.059701,0.011111,0.014706,0,0,...,0.275497,0.090356,0.071478,0,0,0,0,0,0,0
1,3,2.5,0.0,0,0,0.059701,0.011111,0.000000,0,0,...,0.293653,0.090356,0.000000,0,0,0,0,0,0,0


In [46]:
train_allfeature = pd.merge(train_feature,train_attr1,how='left',on='id')

In [48]:
train_allfeature.to_csv('train_allfeature.csv',index=False)

## Test Feature

In [49]:
brandDis = []
colorDis = []
materialDis = []
for i in range(len(test_attr['searchList'])):
    if type(test_attr['searchList'][i]) == str:
        if type(test_attr['brand'][i]) == str:
            brandDis.append(featureGenerate(test_attr['searchList'][i].split(),test_attr['brand'][i].split()))
        else:
            brandDis.append(0)
        if type(test_attr['color'][i]) == str:
            colorDis.append(featureGenerate(test_attr['searchList'][i].split(),test_attr['color'][i].split()))
        else:
            colorDis.append(0)
        if type(test_attr['material'][i]) == str:
            materialDis.append(featureGenerate(test_attr['searchList'][i].split(),test_attr['material'][i].split()))
        else:
            materialDis.append(0)
    else:
        brandDis.append(0)
        colorDis.append(0)
        materialDis.append(0)

In [60]:
test_attr['brandDis'] = pd.Series(brandDis)
test_attr['colorDis'] = pd.Series(colorDis)
test_attr['materialDis'] = pd.Series(materialDis)
test_attr1 = test_attr.iloc[:,[0,11,12,13]]
test_attr1.head(2)

,id,brandDis,colorDis,materialDis
0,1,0,0,0
1,4,0,0,0


In [51]:
test_feature = pd.read_csv('features_tfidf_d_test.csv')

In [63]:
test_allfeature = pd.merge(test_feature,test_attr1,how='left',on='id')

In [64]:
test_allfeature.to_csv('test_allfeature.csv',index=False)